In [4]:
# Workshop Setup
import sys
sys.path.insert(0, '..')

from setup_llm import verify_setup, get_chat_model

verify_setup()

print("\n📚 Session 2, Module 3: Self-Reflection & Quality Patterns")
print("=" * 60)
print("\nThis module covers:")
print("\nPart 1: The Self-Reflection Pattern (15 min)")
print("  • Why single-pass generation fails")
print("  • Generate → Critique → Revise loop")
print("  • Research backing and ROI")
print("\nPart 2: LLM-as-Judge Evaluation (15 min)")
print("  • Structured critique with Pydantic")
print("  • Multi-dimensional scoring rubrics")
print("  • Actionable feedback generation")
print("\nPart 3: Quality Gates & Cost Control (10 min)")
print("  • Threshold-based decisions")
print("  • Iteration limits for cost control")
print("  • Early stopping strategies")
print("\nPart 4: Simple Demo - Report Generator (10 min)")
print("  • End-to-end self-reflection workflow")
print("  • Tracking quality improvements")
print("\n🎯 Build agents that improve their own work!")
print("\n⚠️  Prerequisites: Module 1 (Memory), Module 2 (HITL)")

🔍 Checking LLM Configuration...
📡 Provider: DIAL (Azure OpenAI via EPAM AI Proxy)
✅ DIAL_API_KEY is set

📋 Configuration:
   AZURE_OPENAI_ENDPOINT: https://ai-proxy.lab.epam.com
   AZURE_OPENAI_API_VERSION: 2024-08-01-preview
   AZURE_OPENAI_DEPLOYMENT_NAME: gpt-4

✅ DIAL setup verified successfully!

📚 Session 2, Module 3: Self-Reflection & Quality Patterns

This module covers:

Part 1: The Self-Reflection Pattern (15 min)
  • Why single-pass generation fails
  • Generate → Critique → Revise loop
  • Research backing and ROI

Part 2: LLM-as-Judge Evaluation (15 min)
  • Structured critique with Pydantic
  • Multi-dimensional scoring rubrics
  • Actionable feedback generation

Part 3: Quality Gates & Cost Control (10 min)
  • Threshold-based decisions
  • Iteration limits for cost control
  • Early stopping strategies

Part 4: Simple Demo - Report Generator (10 min)
  • End-to-end self-reflection workflow
  • Tracking quality improvements

🎯 Build agents that improve their own work!

⚠

---

## Part 1: The Self-Reflection Pattern

### Why Single-Pass Generation Fails

**The Problem:**
- LLMs often make errors on first attempt
- No opportunity for self-correction
- Quality varies unpredictably
- Missed edge cases and nuances

**Self-Reflection adds an iterative improvement loop:**

```
┌─────────────────────────────────────────────────────┐
│            SELF-REFLECTION LOOP                     │
├─────────────────────────────────────────────────────┤
│                                                     │
│    Input/Task                                       │
│        │                                            │
│        ▼                                            │
│   ┌──────────┐                                      │
│   │ Generate │ ← Create initial output              │
│   └────┬─────┘                                      │
│        │                                            │
│        ▼                                            │
│   ┌──────────┐                                      │
│   │ Critique │ ← LLM-as-Judge evaluates             │
│   └────┬─────┘                                      │
│        │                                            │
│        ▼                                            │
│   ┌──────────────┐                                  │
│   │ Quality Gate │ ← Score >= threshold?            │
│   └──────┬───────┘                                  │
│          │                                          │
│    ┌─────┴─────┐                                    │
│    ▼           ▼                                    │
│  [Pass]     [Fail]                                  │
│    │           │                                    │
│    │      ┌────▼────┐                               │
│    │      │ Revise  │ ← Apply feedback              │
│    │      └────┬────┘                               │
│    │           └──────→ Back to Critique            │
│    ▼                    (max N iterations)          │
│  Output                                             │
└─────────────────────────────────────────────────────┘
```

### Research Backing

| Study | Finding |
|-------|---------|
| **Reflexion (Shinn et al. 2023)** | 91% on HumanEval (+31% vs GPT-4 baseline) |
| **Self-Refine (Madaan et al. 2023)** | 20-40% improvement across tasks |
| **Constitutional AI (Anthropic)** | 90% reduction in harmful outputs |
| **LinkedIn Text-to-SQL** | 55% → 81% accuracy with reflection |

### Key Components

| Component | Purpose |
|-----------|---------|
| **Generator** | Creates content (code, text, SQL, etc.) |
| **Critic (LLM-as-Judge)** | Evaluates quality with structured rubric |
| **Quality Gate** | Numeric threshold to decide pass/fail |
| **Reviser** | Applies critique feedback to improve output |
| **Iteration Limiter** | Prevents runaway costs (typically max 3) |

### When to Use Self-Reflection

✅ **Use for:**
- Code generation (SQL, Python, etc.)
- Content writing (essays, reports, documentation)
- Complex reasoning tasks
- High-stakes outputs (legal, financial, medical)

❌ **Skip for:**
- Simple lookups or classifications
- Real-time requirements (<100ms)
- Low-stakes casual chat
- Budget-constrained scenarios

---

## Part 2: LLM-as-Judge Evaluation

### The LLM-as-Judge Pattern

Instead of human reviewers, we use an LLM to evaluate outputs using **structured criteria**.

**Key Benefits:**
- Consistent evaluation (same rubric every time)
- Scalable (evaluate thousands of outputs)
- Fast (seconds vs hours for human review)
- Actionable (structured feedback for revision)

### Structured Output for Evaluation

Using Pydantic models ensures consistent, parseable evaluations:

In [5]:
from pydantic import BaseModel, Field
from typing import List

class CritiqueResult(BaseModel):
    """Structured critique from LLM-as-Judge."""
    
    # Multi-dimensional scoring (1-10 each)
    clarity_score: int = Field(
        description="Clarity and readability (1-10)",
        ge=1, le=10
    )
    completeness_score: int = Field(
        description="Coverage of requirements (1-10)",
        ge=1, le=10
    )
    accuracy_score: int = Field(
        description="Factual accuracy (1-10)",
        ge=1, le=10
    )
    overall_score: int = Field(
        description="Overall quality (1-10)",
        ge=1, le=10
    )
    
    # Actionable feedback
    strengths: List[str] = Field(
        description="2-3 specific strengths",
        min_length=1, max_length=3
    )
    weaknesses: List[str] = Field(
        description="2-3 specific areas for improvement",
        min_length=1, max_length=3
    )
    revision_suggestions: List[str] = Field(
        description="Actionable suggestions for next revision",
        min_length=1, max_length=3
    )

print("✅ CritiqueResult model defined")
print("\nStructure:")
print("  • 4 scoring dimensions (1-10 each)")
print("  • Strengths (what's working)")
print("  • Weaknesses (what needs work)")
print("  • Revision suggestions (actionable next steps)")

✅ CritiqueResult model defined

Structure:
  • 4 scoring dimensions (1-10 each)
  • Strengths (what's working)
  • Weaknesses (what needs work)
  • Revision suggestions (actionable next steps)


### Using with_structured_output for Evaluation

The `with_structured_output` method ensures consistent, parseable critiques:

In [6]:
from langchain_core.messages import HumanMessage

# Create a critic using structured output
model = get_chat_model()
critic = model.with_structured_output(CritiqueResult)

# Example content to evaluate
sample_content = """
AI agents are programs that can do tasks. They use LLMs to think.
Agents can be helpful for many things like answering questions.
"""

# Get structured critique
critique = critic.invoke([
    HumanMessage(content=f"""Evaluate this brief explanation of AI agents:

{sample_content}

Rate it for a technical blog audience. Be constructive but honest.""")
])

print("📊 LLM-as-Judge Evaluation:")
print(f"\n   Clarity:      {critique.clarity_score}/10")
print(f"   Completeness: {critique.completeness_score}/10")
print(f"   Accuracy:     {critique.accuracy_score}/10")
print(f"   Overall:      {critique.overall_score}/10")

# Calculate normalized score (0-1)
quality_score = (critique.clarity_score + critique.completeness_score + 
                 critique.accuracy_score + critique.overall_score) / 40
print(f"\n   🎯 Quality Score: {quality_score:.1%}")

print(f"\n✨ Strengths:")
for s in critique.strengths:
    print(f"   • {s}")

print(f"\n⚠️  Weaknesses:")
for w in critique.weaknesses:
    print(f"   • {w}")

print(f"\n📝 Revision Suggestions:")
for r in critique.revision_suggestions:
    print(f"   • {r}")

📊 LLM-as-Judge Evaluation:

   Clarity:      6/10
   Completeness: 4/10
   Accuracy:     5/10
   Overall:      5/10

   🎯 Quality Score: 50.0%

✨ Strengths:
   • The explanation is concise and easy to understand.
   • It introduces the concept of AI agents and their utility.

⚠️  Weaknesses:
   • The description lacks technical depth and specificity.
   • The role of LLMs in AI agents is oversimplified.

📝 Revision Suggestions:
   • Expand on the definition of AI agents, including their components and functionality.
   • Provide examples of tasks AI agents can perform and how LLMs contribute to their capabilities.
   • Clarify the relationship between AI agents and LLMs, emphasizing their integration and limitations.


---

## Part 3: Simple Demo - Self-Improving Report Generator

Let's build a complete self-reflection workflow for generating technical summaries.

### What We'll Build

1. **Generator Node**: Creates initial summary
2. **Critic Node**: LLM-as-Judge evaluation
3. **Quality Gate**: Pass/fail routing
4. **Reviser**: Improves based on feedback (loops back to Generator)
5. **Finalizer**: Outputs result with improvement metrics

In [8]:
from typing import TypedDict, Literal, List
from langgraph.graph import StateGraph, START, END

# ═══════════════════════════════════════════════════════════════════════════
# STATE DEFINITION
# ═══════════════════════════════════════════════════════════════════════════

class ReflectionState(TypedDict):
    """State for self-reflection workflow."""
    
    # Input
    topic: str
    requirements: str
    
    # Generated content
    content: str
    
    # Critique
    critique: dict
    quality_score: float  # 0.0 to 1.0
    
    # Control
    iteration: int
    max_iterations: int
    threshold: float
    
    # History (for tracking improvement)
    score_history: List[float]

print("✅ ReflectionState defined")
print("\nKey fields:")
print("  • topic, requirements: Input")
print("  • content: Generated output (updated each revision)")
print("  • critique, quality_score: Evaluation results")
print("  • iteration, max_iterations, threshold: Control flow")
print("  • score_history: Track improvement over iterations")

✅ ReflectionState defined

Key fields:
  • topic, requirements: Input
  • content: Generated output (updated each revision)
  • critique, quality_score: Evaluation results
  • iteration, max_iterations, threshold: Control flow
  • score_history: Track improvement over iterations


In [9]:
# ═══════════════════════════════════════════════════════════════════════════
# NODE FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════

def generate(state: ReflectionState) -> dict:
    """Generate or revise content based on feedback."""
    
    model = get_chat_model()
    iteration = state.get("iteration", 0) + 1
    
    print(f"\n{'='*60}")
    print(f"📝 GENERATE - Iteration {iteration}")
    print(f"{'='*60}")
    
    if state.get("critique"):  # Revision mode
        critique = state["critique"]
        prompt = f"""Revise this content based on feedback:

TOPIC: {state['topic']}
REQUIREMENTS: {state['requirements']}

CURRENT CONTENT:
{state['content']}

FEEDBACK TO ADDRESS:
Weaknesses: {critique.get('weaknesses', [])}
Suggestions: {critique.get('revision_suggestions', [])}

Write an IMPROVED version addressing all feedback."""
        print("   Mode: REVISION (applying feedback)")
    else:  # Initial generation
        prompt = f"""Write a clear, concise summary on this topic:

TOPIC: {state['topic']}
REQUIREMENTS: {state['requirements']}

Provide a well-structured, informative summary."""
        print("   Mode: INITIAL GENERATION")
    
    response = model.invoke([HumanMessage(content=prompt)])
    content = response.content.strip()
    
    print(f"   Generated {len(content)} characters")
    
    return {
        "content": content,
        "iteration": iteration
    }

print("✅ generate node defined")

✅ generate node defined


In [10]:
def critique(state: ReflectionState) -> dict:
    """LLM-as-Judge evaluation with structured output."""
    
    model = get_chat_model()
    critic = model.with_structured_output(CritiqueResult)
    
    print(f"\n{'='*60}")
    print(f"🔍 CRITIQUE - LLM-as-Judge")
    print(f"{'='*60}")
    
    result = critic.invoke([
        HumanMessage(content=f"""Evaluate this content rigorously:

TOPIC: {state['topic']}
REQUIREMENTS: {state['requirements']}

CONTENT TO EVALUATE:
{state['content']}

Provide honest, constructive feedback with specific examples.""")
    ])
    
    # Calculate normalized quality score
    scores = [result.clarity_score, result.completeness_score,
              result.accuracy_score, result.overall_score]
    quality_score = sum(scores) / 40  # Normalize to 0-1
    
    critique_dict = {
        "clarity": result.clarity_score,
        "completeness": result.completeness_score,
        "accuracy": result.accuracy_score,
        "overall": result.overall_score,
        "strengths": result.strengths,
        "weaknesses": result.weaknesses,
        "revision_suggestions": result.revision_suggestions
    }
    
    print(f"\n   📊 Scores: Clarity={result.clarity_score}, Completeness={result.completeness_score}")
    print(f"              Accuracy={result.accuracy_score}, Overall={result.overall_score}")
    print(f"   🎯 Quality Score: {quality_score:.1%} (threshold: {state['threshold']:.1%})")
    
    # Track score history
    history = state.get("score_history", [])
    history.append(quality_score)
    
    return {
        "critique": critique_dict,
        "quality_score": quality_score,
        "score_history": history
    }

print("✅ critique node defined")

✅ critique node defined


In [11]:
def should_continue(state: ReflectionState) -> Literal["revise", "finalize"]:
    """Quality gate: decide whether to continue or finalize."""
    
    print(f"\n{'='*60}")
    print(f"🚦 QUALITY GATE")
    print(f"{'='*60}")
    print(f"   Score: {state['quality_score']:.1%} | Threshold: {state['threshold']:.1%}")
    print(f"   Iteration: {state['iteration']}/{state['max_iterations']}")
    
    # Check quality threshold
    if state["quality_score"] >= state["threshold"]:
        print(f"   ✅ PASS: Quality threshold met!")
        return "finalize"
    
    # Check iteration limit
    if state["iteration"] >= state["max_iterations"]:
        print(f"   ⚠️  MAX ITERATIONS: Cost control triggered")
        return "finalize"
    
    print(f"   🔄 CONTINUE: Sending for revision")
    return "revise"

print("✅ should_continue routing defined")

✅ should_continue routing defined


In [12]:
def finalize(state: ReflectionState) -> dict:
    """Output final content with improvement metrics."""
    
    history = state.get("score_history", [])
    
    print(f"\n{'='*60}")
    print(f"✅ FINALIZE - Self-Reflection Complete")
    print(f"{'='*60}")
    
    # Calculate improvement
    if len(history) >= 2:
        initial = history[0]
        final = history[-1]
        improvement = final - initial
        print(f"\n   📈 IMPROVEMENT METRICS:")
        print(f"      Initial: {initial:.1%}")
        print(f"      Final:   {final:.1%}")
        print(f"      Gain:    {improvement:+.1%}")
    
    print(f"\n   📊 Quality Progression:")
    for i, score in enumerate(history, 1):
        bar = "█" * int(score * 20)
        print(f"      Iteration {i}: {bar} {score:.1%}")
    
    print(f"\n   📝 Final Content ({len(state['content'])} chars):")
    print(f"   {'-'*50}")
    preview = state['content'][:400] + "..." if len(state['content']) > 400 else state['content']
    print(f"   {preview}")
    
    return {}

print("✅ finalize node defined")

✅ finalize node defined


In [13]:
# ═══════════════════════════════════════════════════════════════════════════
# BUILD THE GRAPH
# ═══════════════════════════════════════════════════════════════════════════

builder = StateGraph(ReflectionState)

# Add nodes
builder.add_node("generate", generate)
builder.add_node("critique", critique)
builder.add_node("finalize", finalize)

# Add edges
builder.add_edge(START, "generate")
builder.add_edge("generate", "critique")

# Quality gate: conditional routing after critique
builder.add_conditional_edges(
    "critique",
    should_continue,
    {
        "revise": "generate",   # Loop back for improvement
        "finalize": "finalize"  # Exit to finalization
    }
)

builder.add_edge("finalize", END)

# Compile
reflection_graph = builder.compile()

print("✅ Self-Reflection Graph Compiled!")
print("""
Graph Structure:

    START
      │
      ▼
  ┌──────────┐
  │ generate │ ◄─────────────────┐
  └────┬─────┘                   │
       │                         │
       ▼                         │
  ┌──────────┐                   │
  │ critique │                   │
  └────┬─────┘                   │
       │                         │
       ▼                         │
  ┌───────────┐                  │
  │  quality  │                  │
  │   gate    │                  │
  └─────┬─────┘                  │
        │                        │
   ┌────┴────┐                   │
   ▼         ▼                   │
 [pass]   [fail]─────────────────┘
   │       (revise)
   ▼
  ┌──────────┐
  │ finalize │
  └────┬─────┘
       │
       ▼
      END
""")

✅ Self-Reflection Graph Compiled!

Graph Structure:

    START
      │
      ▼
  ┌──────────┐
  │ generate │ ◄─────────────────┐
  └────┬─────┘                   │
       │                         │
       ▼                         │
  ┌──────────┐                   │
  │ critique │                   │
  └────┬─────┘                   │
       │                         │
       ▼                         │
  ┌───────────┐                  │
  │  quality  │                  │
  │   gate    │                  │
  └─────┬─────┘                  │
        │                        │
   ┌────┴────┐                   │
   ▼         ▼                   │
 [pass]   [fail]─────────────────┘
   │       (revise)
   ▼
  ┌──────────┐
  │ finalize │
  └────┬─────┘
       │
       ▼
      END



In [15]:
# ═══════════════════════════════════════════════════════════════════════════
# RUN THE SELF-REFLECTION WORKFLOW
# ═══════════════════════════════════════════════════════════════════════════

print("🚀 Running Self-Reflection Report Generator")
print("=" * 60)

result = reflection_graph.invoke({
    "topic": "Benefits of LangGraph for Production AI Agents",
    "requirements": """
- 150-200 words
- Include 3 specific benefits
- Mention state management and human-in-the-loop
- Professional tone for engineering blog
""",
    "content": "",
    "critique": {},
    "quality_score": 0.0,
    "iteration": 0,
    "max_iterations": 3,
    "threshold": 0.75,  # 75% quality required
    "score_history": []
})

print("\n" + "=" * 60)
print("🎯 SELF-REFLECTION COMPLETE!")
print("=" * 60)

🚀 Running Self-Reflection Report Generator

📝 GENERATE - Iteration 1
   Mode: INITIAL GENERATION
   Generated 1429 characters

🔍 CRITIQUE - LLM-as-Judge

   📊 Scores: Clarity=9, Completeness=8
              Accuracy=9, Overall=9
   🎯 Quality Score: 87.5% (threshold: 75.0%)

🚦 QUALITY GATE
   Score: 87.5% | Threshold: 75.0%
   Iteration: 1/3
   ✅ PASS: Quality threshold met!

✅ FINALIZE - Self-Reflection Complete

   📊 Quality Progression:
      Iteration 1: █████████████████ 87.5%

   📝 Final Content (1429 chars):
   --------------------------------------------------
   **Benefits of LangGraph for Production AI Agents**

LangGraph is a powerful framework designed to enhance the development and deployment of production-grade AI agents. By addressing key challenges in scalability, reliability, and adaptability, LangGraph offers several distinct advantages for engineering teams building AI-driven systems.

First, LangGraph excels in **state management**, enabling AI...

🎯 SELF-REFLECTION 

---

## Key Takeaways

### Self-Reflection Pattern

| Component | Purpose | Implementation |
|-----------|---------|----------------|
| **Generator** | Creates initial output | Single node, switches between generate/revise |
| **Critic (LLM-as-Judge)** | Evaluates quality | `with_structured_output(CritiqueResult)` |
| **Quality Gate** | Pass/fail decision | Conditional edge with threshold check |
| **Iteration Limiter** | Cost control | `max_iterations` parameter |

### Production Tips

1. **Start Conservative**: Begin with threshold=0.7, max_iterations=3
2. **Track Improvement**: Log score_history to validate the pattern helps
3. **Cost Awareness**: Each iteration costs ~$0.008 (GPT-4o)
4. **Diminishing Returns**: Most gains happen in first 2 iterations

### When to Use

| Use Case | Recommended? | Reason |
|----------|--------------|--------|
| Content generation | ✅ Yes | High ROI, visible quality improvement |
| Code generation | ✅ Yes | Catches bugs, improves structure |
| Simple Q&A | ❌ No | Overkill, adds latency |
| Real-time chat | ❌ No | Too slow for conversational UX |

---

## What's Next?

### Module 4: Multi-Agent Collaboration

Build on self-reflection by adding **multiple specialized agents**:
- Hierarchical agents (supervisor → workers)
- Peer review patterns (proposer ↔ reviewer)
- Consensus building across agents
- Send API for dynamic parallelism

### Connection to Project

The **Incident PostMortem Generator** project combines:
- Self-reflection for report quality (this module!)
- Multiple agents: Log Analyzer, Root Cause Agent, Report Writer
- Human review checkpoints
- Production streaming output